In [69]:
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy.tokenizer import Tokenizer
from scipy.stats import ttest_ind
from scipy import spatial, stats
import os

Topic numbers of target topics, acquirer topics and neutral topics

In [70]:
tt= sorted([40,22,38,1,28,13,41,14,35,49,36,29,33,5,3,34])
at = sorted([46,42,10,37,7,27,18,19,23,39,32,15,43,17,2,25,47])
nt = sorted([6,8,30,11,48,16,12,24,26,20,31,50,21,4,44,9,45])

All combinations from the network in order target-neutral-acquirer

In [71]:
nt_tt = list()
for t1 in nt:
    for t2 in tt:
        nt_tt.append([t1, t2])
nt_at = list()
for t1 in nt:
    for t2 in at:
        nt_at.append([t1, t2])

In [72]:
all_combis = list()
for t1 in nt_tt:
    for t2 in nt_at:
        t = [t1[1], t1[0], t2[1]]
        if t not in all_combis:
            all_combis.append(t)

In [73]:
len(all_combis)

4624

Read in the meta data an theta matrix in dataframes and reset their indices.

In [74]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [76]:
path = find('meta.csv', 'C:/')
data = pd.read_csv(path)
path = find('thetamatrix50.csv','C:/')
theta = pd.read_csv(path)
data = data.reset_index(drop=True)
theta = theta.reset_index(drop=True)

C:\Users\20150624\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Create a dataframe that reports for every pair of targets and acquirers the median topic proportions over all their documents, if the document was before or after the effective date, the similarity between the target and acquirer vector and their industries.

In [127]:
proportions = pd.DataFrame(columns=['Target', 'Acquiror', 'TargetTopics', 'AcquirorTopics', 'Period', 'Similarity'])
nlp = English()
tokenizer = Tokenizer(nlp.vocab)
target_text_length = list()
acquirer_text_length = list()
before = list()
after = list()
valid_pair_before = list()
valid_pair_after = list()
same_industry = list()
diff_industry = list()
for company in data['companydate'].unique():
    target = list(data[data['companydate']==company]['Target.Name'])[0]
    acquirer = list(data[data['companydate']==company]['Acquiror.Name'])[0]
    target_industry = list(data[data['Target.Name']==target]['Target.Macro.Industry'])[0]
    acquiror_industry = list(data[data['Acquiror.Name']==acquirer]['Acquiror.Macro.Industry'])[0]    
    year = list(data[data['companydate']==company]['DealYear'])[0]
    docyear = list(data[data['companydate']==company]['DocumentYear'])[0]
    pair = (target, acquirer)
    period =list(data[data['companydate']==company]['period'])[0]
    target_topics= list()
    acquiror_topics = list()
    if (period == 'before') and (pair not in before):
        before.append(pair)
        indices = data[(data['Target.Name']==target)&(data['Acquiror.Name']==acquirer)&(data['period']=='before')].index
        target_indices = list()
        acquirer_indices = list()
        target_length = 0
        acquirer_length = 0
        for index in indices:
            if data.iloc[index]['companytype'] == 'Target':
                target_indices.append(index)
            if data.iloc[index]['companytype'] ==  'Acquiror':
                acquirer_indices.append(index)
        for index in target_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            target_length += len(tokens)
        for index in acquirer_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            acquirer_length += len(tokens)
        target_text_length.append(target_length)
        acquirer_text_length.append(acquirer_length)
        if target_length >=162 and acquirer_length >=162:
            valid_pair_before.append(pair)
            for column in theta.columns:
                if column != 'Unnamed: 0':
                    target_topics.append(theta.iloc[target_indices][column].median())
                    acquiror_topics.append(theta.iloc[acquirer_indices][column].median())
            sum_target_topics=sum(target_topics)
            target_topics = [i/sum_target_topics for i in target_topics]
            sum_acquiror_topics = sum(acquiror_topics)
            acquiror_topics = [i/sum_acquiror_topics for i in acquiror_topics]
            similarity = 1-spatial.distance.euclidean(target_topics, acquiror_topics)
            if target_industry == acquiror_industry:
                same_industry.append(similarity)
            if target_industry != acquiror_industry:
                diff_industry.append(similarity)
            props = pd.DataFrame({'Target':target,'Acquiror':acquirer,'TargetTopics':[target_topics],'AcquirorTopics':[acquiror_topics],'Period':period,
                                 'Target industry':target_industry,'Acquirer industry':acquiror_industry,
                                 'Similarity':similarity,'Year':int(year), 'DocYear':int(docyear)})
            proportions = proportions.append(props)

    if (period == 'after') and (pair not in after):
        after.append(pair)
        indices = data[(data['Target.Name']==target)&(data['Acquiror.Name']==acquirer)&(data['period']=='after')].index
        target_indices = list()
        acquirer_indices = list()
        target_length = 0
        acquirer_length = 0
        for index in indices:
                if data.iloc[index]['companytype'] == 'Target':
                    target_indices.append(index)
                if data.iloc[index]['companytype'] ==  'Acquiror':
                    acquirer_indices.append(index)
        for index in target_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            target_length += len(tokens)
        for index in acquirer_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            acquirer_length += len(tokens)
        target_text_length.append(target_length)
        acquirer_text_length.append(acquirer_length)
        if target_length >=162 and acquirer_length >=162:
            valid_pair_after.append(pair)
            for column in theta.columns:
                if column != 'Unnamed: 0':
                    target_topics.append(theta.iloc[target_indices][column].median())
                    acquiror_topics.append(theta.iloc[acquirer_indices][column].median())
            sum_target_topics=sum(target_topics)
            target_topics = [i/sum_target_topics for i in target_topics]
            sum_acquiror_topics = sum(acquiror_topics)
            acquiror_topics = [i/sum_acquiror_topics for i in acquiror_topics]                    
            similarity = 1-spatial.distance.euclidean(target_topics, acquiror_topics)
            if target_industry == acquiror_industry:
                same_industry.append(similarity)
            if target_industry != acquiror_industry:
                diff_industry.append(similarity)
            props = pd.DataFrame({'Target':target,'Acquiror':acquirer,'TargetTopics':[target_topics],'AcquirorTopics':[acquiror_topics],'Period':period,
                                 'Target industry':target_industry,'Acquirer industry':acquiror_industry,
                                  'Similarity':similarity, 'Year':int(year),'DocYear':int(docyear)})
            proportions = proportions.append(props)
proportions = proportions.reset_index(drop=True)

Read in the correlations between topics and drop an unnecessary column.

In [128]:
path = find('positivecorrelationstopics.csv', 'C:/')
relations = pd.read_csv(path)
relations = relations.drop(labels='Unnamed: 0', axis=1)

Create a dataframe with a column for every target-neutral-acquirer pattern.

In [139]:
tan = pd.DataFrame(columns=['Target', 'Acquiror', 'TargetTopics', 'AcquirorTopics', 'Period'])
all_combinations = [str(i) for i in all_combis]
all_combinations = pd.DataFrame(columns=all_combinations)
tan = pd.concat([tan,all_combinations], axis=1)

In [140]:
len(all_combis)

4624

For each deal, count the sizes of the target-neutral-acquirer patterns.

In [145]:
allframes = pd.DataFrame()
all_combinations = [str(i) for i in all_combis]
for index, row in proportions.iterrows():
    target_topics = row['TargetTopics']
    acquirer_topics = row['AcquirorTopics']
    combis = dict.fromkeys(all_combinations)
    for combi in all_combis:
        relation = 0
        t = target_topics[combi[0]-1]
        afort = acquirer_topics[combi[0]-1]
        n = target_topics[combi[1]-1]
        tn = relations.iloc[combi[0]-1][combi[1]-1]
        n2= acquirer_topics[combi[1]-1]
        a= acquirer_topics[combi[2]-1]
        tfora = target_topics[combi[2]-1]
        an = relations.iloc[combi[2]-1][combi[1]-1]
        ta = relations.iloc[combi[0]-1][combi[2]-1]
        tn = (t+n-afort)*tn
        an = (a+n-tfora)*an
      
        if tn>0 and an>0:
            relation = tn+an
            if ta > 0.01: 
                relation = relation - ((afort+tfora)*ta)
        combis[str(combi)] = relation
    if row['Target industry'] == row['Acquirer industry']:
        ind = 'Same'
    elif row['Target industry'] != row['Acquirer industry']:
        ind = "Different"
    tan = pd.DataFrame({'Target':row['Target'],'Acquiror':row['Acquiror'],'TargetTopics':t,'AcquirorTopics':a,
                        'Period':row['Period'], 'TargetIndustry':row['Target industry'], 
                        'AcquirerIndustry':row['Acquirer industry'], 'Industry':ind}, index=[index])
    all_comb = pd.DataFrame(combis, index=[index])
    frame = pd.concat([tan,all_comb], axis=1)
    allframes = allframes.append(frame)

Topic numbers of the target-neutral-acquirer patterns that are visible in the network in the thesis.

In [147]:
strong_relations = [[22,9,37],[22,31,37],[22,31,46],[3,31,37],[3,31,46],[3,20,46],[33,20,46],[3,44,25],[3,44,2],[33,44,25],[33,44,2],
[3,45,2],[3,45,27],[3,45,2],[35,26,32],[1,26,32],[49,26,32],[29,48,32],[49,48,32],[13,48,32],[49,30,32],[13,30,32],[28,30,32]]

In [148]:
strong_relations = [str(i) for i in strong_relations]

Get the target-neutral-acquirer patterns with a mean column value above 0. Also count how many of the patterns from the network have a mean above 0.

In [151]:
nonzero = dict()
strong = 0
for column in allframes.columns:
    if column not in ['Target', 'Acquiror', 'TargetTopics','AcquirorTopics','Period','Total','Industry', 
                     'TargetIndustry','AcquirerIndustry']:
        if allframes[str(column)].mean() >0:
            nonzero[str(column)] = allframes[str(column)].mean()
            if str(column) in strong_relations:
                strong+=1

In [152]:
print(len(nonzero))
strong

71


22

Sort the patterns from largest to smallest.

In [153]:
{k: v for k, v in sorted(nonzero.items(), key=lambda item: item[1], reverse=True)}

{'[3, 31, 37]': 0.009368240294520904,
 '[3, 31, 46]': 0.009216920224667758,
 '[3, 44, 25]': 0.007132308604592227,
 '[3, 44, 2]': 0.0058335578273557755,
 '[1, 26, 32]': 0.00521004091953755,
 '[3, 45, 27]': 0.005184929360257226,
 '[35, 26, 32]': 0.005138629660869657,
 '[3, 45, 32]': 0.004305718341250957,
 '[49, 26, 32]': 0.00416738156668455,
 '[3, 45, 2]': 0.003471911221703301,
 '[3, 45, 37]': 0.0033310648991675576,
 '[3, 45, 43]': 0.0032901048240471637,
 '[13, 26, 32]': 0.0030801350626048444,
 '[36, 45, 27]': 0.003034277545848743,
 '[35, 30, 32]': 0.0027325960700973284,
 '[22, 31, 37]': 0.0026133278712643627,
 '[29, 26, 32]': 0.002562625588691844,
 '[22, 31, 46]': 0.0025564068047307315,
 '[14, 8, 2]': 0.0023162003105620375,
 '[33, 44, 25]': 0.0021999181830636297,
 '[36, 45, 32]': 0.0021936596839091252,
 '[14, 8, 46]': 0.002160369714786622,
 '[14, 8, 43]': 0.0020492158697137377,
 '[3, 8, 2]': 0.0017716298389512685,
 '[3, 8, 43]': 0.0017481803971489951,
 '[3, 8, 46]': 0.001668628945292432

In [154]:
allframes = allframes.fillna(value=0)

For each deal, count the total size of their target-neutral-acquirer patterns.

In [155]:
allframes['Total']= ""
for index, row in allframes.iterrows():
    total = 0
    target= row['Target']
    acquirer = row['Acquiror']
    period = row['Period']
    subset = allframes[(allframes['Target']==target)&(allframes['Acquiror']==acquirer)&(allframes['Period']==period)]
    for column in subset.columns:
        if column not in ['Target', 'Acquiror','TargetTopics','AcquirorTopics','Period', 'Total', 'TargetIndustry','AcquirerIndustry','Industry']:
            if row[column] != 0:
                value = row[column]
                total += value 
    allframes.at[index,'Total'] = total

In [156]:
allframes['Total'].mean()

0.12830443586266285

In [157]:
allframes

,Target,Acquiror,TargetTopics,AcquirorTopics,Period,TargetIndustry,AcquirerIndustry,Industry,"[1, 4, 2]","[1, 4, 7]",...,"[49, 50, 25]","[49, 50, 27]","[49, 50, 32]","[49, 50, 37]","[49, 50, 39]","[49, 50, 42]","[49, 50, 43]","[49, 50, 46]","[49, 50, 47]",Total
0,Carolina Crescent Health Plan,Absolute Total Care,0.000139,0.000059,before,Healthcare,Healthcare,Same,0,0,...,0,0,0,0,0,0,0,0,0,0.051354
1,Specialty Therapeutic Hldg Inc,Centene Corp,0.000557,0.000087,before,Retail,Healthcare,Different,0,0,...,0,0,0,0,0,0,0,0,0,0.155446
2,Access Dental Plan LLC,DentaQuest LLC,0.000010,0.000378,before,Healthcare,Healthcare,Same,0,0,...,0,0,0,0,0,0,0,0,0,0.325306
3,Integra Partners LLC,Tufts Assocd Health Plans Inc,0.003185,0.002665,before,Financials,Healthcare,Different,0,0,...,0,0,0,0,0,0,0,0,0,0.0848456
4,Healthagen LLC,Aetna Inc,0.000401,0.011287,before,High Technology,Healthcare,Different,0,0,...,0,0,0,0,0,0,0,0,0,0.0942474
5,Aetna Inc-Missouri Medicaid,WellCare Health Plans Inc,0.000664,0.000407,before,Healthcare,Healthcare,Same,0,0,...,0,0,0,0,0,0,0,0,0,0.12042
6,Coventry Health Care Inc,Aetna Inc,0.002816,0.003249,before,Healthcare,Healthcare,Same,0,0,...,0,0,0,0,0,0,0,0,0,0.065171
7,bswift LLC,Aetna Inc,0.001983,0.015530,before,High Technology,Healthcare,Different,0,0,...,0,0,0,0,0,0,0,0,0,0.205654
8,American Health Network Inc,UnitedHealth Group Inc,0.001429,0.049769,before,Healthcare,Healthcare,Same,0,0,...,0,0,0,0,0,0,0,0,0,0.0734731
9,AIG Sigorta AS,GIG,0.000915,0.010816,before,Healthcare,Financials,Different,0,0,...,0,0,0,0,0,0,0,0,0,0.0462706


In [158]:
allframessame = allframes[allframes['Industry']=='Same']
allframesdif = allframes[allframes['Industry']=='Different']

In [159]:
print(allframessame['Total'].mean())
print(allframesdif['Total'].mean())

0.11982066141183875
0.1376451572277115


In [160]:
ttest_ind(allframessame['Total'],allframesdif['Total'])

Ttest_indResult(statistic=-0.6310932243098103, pvalue=0.5286791590866475)

One example deal with subset before and after the effective date.

In [162]:
subsetbef = allframes[(allframes['Target']=='All Points EAP & Orgal Svcs')&(allframes['Period']=='before')]
subsetaft = allframes[(allframes['Target']=='All Points EAP & Orgal Svcs')&(allframes['Period']=='after')]
before = dict()
after = dict()
for column in subsetbef:
    if column not in ['Target','Acquiror','TargetTopics','AcquirorTopics','Period','Total','TargetIndustry','AcquirerIndustry','Industry']:
        if subsetbef[column].values[0] >0 :
            before[column]=subsetbef[column].values[0]
for column in subsetaft:
    if column not in ['Target','Acquiror','TargetTopics','AcquirorTopics','Period','Total','TargetIndustry','AcquirerIndustry','Industry']:
        if subsetaft[column].values[0] >0 :
            after[column]=subsetaft[column].values[0]

For the example deal, compute the differences in size for each pattern before and after the effective date.

In [163]:
differences = dict()
for key in before.keys():
    if key in after.keys():
        differences[key]=after[key]-before[key]
    else:
        differences[key] = -before[key]
for key in after.keys():
    if key not in before.keys():
        differences[key] = after[key]

Sort the patterns for the example deal by largest difference in pattern size before compared to after the effective date.

In [81]:
{k:v for k, v in sorted(differences.items(),  key=lambda item: item[1], reverse=True)}

{'[13, 26, 32]': 0.0079914247560128,
 '[13, 30, 32]': 0.007483172503434258,
 '[13, 48, 32]': 0.007469608590594442,
 '[1, 26, 32]': 0.0034781856101880134,
 '[35, 26, 32]': 0.0033114073351546286,
 '[49, 26, 32]': 0.0031649060166019935,
 '[29, 26, 32]': 0.0030060084897712467,
 '[49, 48, 32]': 0.0025359290123600194,
 '[35, 30, 32]': 0.0023534237513149446,
 '[28, 48, 32]': 0.002288533518554108,
 '[29, 48, 32]': 0.002282348874754694,
 '[13, 50, 18]': 0.0021838772426695198,
 '[13, 50, 47]': 0.002128346093988027,
 '[36, 45, 32]': 0.0020869262913204723,
 '[1, 30, 32]': 0.0020816729750660666,
 '[49, 30, 32]': 0.0020810610831659567,
 '[28, 30, 32]': 0.002035572786545834,
 '[5, 30, 32]': 0.002034764449515972,
 '[29, 30, 32]': 0.002002802916477223,
 '[13, 50, 39]': 0.0019212302467865906,
 '[14, 8, 46]': 0.001497618394609137,
 '[14, 8, 43]': 0.0014506459805620087,
 '[13, 24, 37]': 0.001098502133681299,
 '[13, 24, 25]': 0.0010733192806360648,
 '[33, 20, 25]': 0.0006813214083640014,
 '[33, 20, 46]': 0

Create subset before and after the effective dates.

In [ ]:
allframesbefore = allframes[allframes['Period']=='before']
allframesafter = allframes[allframes['Period']=='after']

In [ ]:
print(allframesbefore['Total'].mean())
print(allframesafter['Total'].mean())

Create a dataframe with complete data before and after the effective date.

In [ ]:
allframestime = pd.DataFrame(columns=['Target', 'Acquirer', 'Before', 'After'])
for index, row in allframesbefore.iterrows():
    target = row['Target']
    acquirer = row['Acquiror']
    before = row['Total']
    if len(allframesafter[(allframesafter['Target']==target) & (allframesafter['Acquiror']==acquirer)])==1:
        after = allframesafter[(allframesafter['Target']==target) & (allframesafter['Acquiror']==acquirer)]['Total'].values[0]
        value = pd.DataFrame({'Target':target, 'Acquirer':acquirer, 'Before':before,'After':after}, index=[index])
        allframestime = allframestime.append(value)

T test to check if the pattern sizes before differed significantly from the sizes after.

In [ ]:
ttest_ind(allframestime['Before'], allframestime['After'])

In [58]:
allframes.to_csv('C:/tanpatterns.csv')

Compute the complementarity patterns between a target and the mean of all other acquirers and an acquirer and the mean of all other targets. 

In [ ]:
other_complementarities = pd.DataFrame(columns=['Target', 'Acquirer', 'Period', 'Complementarity', 'Mean Complementarity Other Acquirers', 'Mean Complementarity Other Targets'])
complementarity_highest = 0
other_acquirers_higher =0
other_targets_higher = 0
for index, row in allframes.iterrows():
    other_com_target = list()
    other_com_acquirer = list()
    complementarity = row['Complementarity']
    acquirer_topics = row['AcquirerTopics']
    target_topics = row['TargetTopics']
    period = row['Period']
    for index2, row2 in complementarities.iterrows():
        if period == row2['Period']:
            ttcomplementarity = 0
            atcomplementarity = 0
            if index != index2:
                at = row2['AcquirerTopics']
                tt = row2['TargetTopics']
                for topic in acquirer_topics:
                    correlations = relations.iloc[acquirer_topics.index(topic)]
                    for topic2 in tt:
                        i1 = acquirer_topics.index(topic)
                        i2 = tt.index(topic2)
                        if i1 != i2:
                            correlation = correlations[tt.index(topic2)]
                            targetforthattopic = tt[i1]
                            acquirerforthattopic = acquirer_topics[i2]
                            if topic > targetforthattopic and topic2 > acquirerforthattopic:
                                ttcomplementarity += topic*correlation + topic2*correlation
                            elif targetforthattopic > topic and acquirerforthattopic >topic2:
                                ttcomplementarity += topic*correlation + topic2*correlation
                for topic in target_topics:
                    correlations = relations.iloc[target_topics.index(topic)]
                    for topic2 in at:
                        i1 = target_topics.index(topic)
                        i2 = at.index(topic2)
                        if i1 != i2:
                            correlation = correlations[at.index(topic2)]
                            acquirerforthattopic = at[i1]
                            targetforthattopic = target_topics[i2]
                            if topic > acquirerforthattopic and topic2 > targetforthattopic:
                                atcomplementarity += topic*correlation + topic2*correlation
                            elif acquirerforthattopic > topic and targetforthattopic >topic2:
                                atcomplementarity += topic*correlation + topic2*correlation                        
            other_com_target.append(ttcomplementarity)
            other_com_acquirer.append(atcomplementarity)
    complement = pd.DataFrame({'Target':row['Target'],'TargetIndustry':row['TargetIndustry'], 'Acquirer': row['Acquirer'],'Period':row['Period'],
                               'AcquirerIndustry':row['AcquirerIndustry'],'Complementarity':complementarity,'Mean Complementarity Other Acquirers':mean(other_com_acquirer),
                              'Mean Complementarity Other Targets':mean(other_com_target),'Industry':row['Industry']}, index=[index])
    other_complementarities = other_complementarities.append(complement)